In [1]:
from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from pyecharts.datasets import register_url
import pandas as pd
import numpy as np
import time

In [2]:
 from pyecharts.globals import CurrentConfig, OnlineHostType

 CurrentConfig.ONLINE_HOST = OnlineHostType.NOTEBOOK_HOST

In [3]:

df_global_confirm = pd.read_csv('C:/Users/MinkeyJiang/Desktop/722/old_database/time_series_covid19_confirmed_global.csv')
# change the country name same to packages
def replace_country_name(val):
    replace_dic = {
        'US':'United States',
        'Congo (Brazzaville)':'Congo',
        'Congo (Kinshasa)':'Dem. Rep. Congo',
        'South Sudan':'S. Sudan',
        'Central African Republic':'Central African Rep.',
        'Western Sahara':'W. Sahara',
        'Laos':'Lao PDR',
        'Korea, South':'Korea'
    }
    for k, v in replace_dic.items():
        val = val.replace(k,v)
    return val

df_global_confirm.iloc[:,1] = df_global_confirm.iloc[:,1].apply(replace_country_name)
# number of confirm for each country
df_global_country_confirm = df_global_confirm.groupby(['Country/Region']).sum().reset_index()

In [4]:
geo = Geo()
data_pair, data_pair_high = [], []
for idx, row in df_global_confirm.iloc[:,[0,1,2,3,-1]].iterrows():
    if int(row[-1]) > 1e6:
        label = row['Country/Region'] if row['Province/State'] is np.nan  else row['Province/State']
        geo.add_coordinate(label, row['Long'], row['Lat'])
        data_pair_high.append([label, int(row[-1])])
        
for idx, row in df_global_country_confirm.iloc[:,[0,-1]].iterrows():
    data_pair.append([row['Country/Region'], int(row[-1])])

    # label for geo and bar
fmt_js = """function (params) {return params.name+':'+Number(params.value[2]);}"""


In [5]:
"""
Bar:
show the daliy top 10 country
leftbottom
"""
t = sorted(data_pair, key=lambda x: x[1], reverse=True)[:10][::-1]

bar = Bar()
bar.add_xaxis([x for x, y in t])
bar.add_yaxis("", [y for x, y in t],
              itemstyle_opts={'normal': {'borderColor': '#115692',
                                         'shadowColor': '#115692',
                                         'shadowBlur': 20,
                                         'opacity': 0.8
                                         }
                              })
# Label format setting
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=10,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
# Global format setting
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, max_=1e7),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="TOP 10", pos_top='55%', pos_left='5%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=1e6,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['rgba(219,112,147,0.4)', 'rgba(238,25,27,0.9)', 'rgba(238,25,27,1)']))
bar.reversal_axis()

"""
map
"""
mp = Map()
mp.add(
    "Cumulative confirmed global",
    data_pair,
    "world",
    zoom=1.2,
    is_map_symbol_show=False,
    is_roam=False)

mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                   itemstyle_opts={'normal': {
                       'areaColor': '#071126',
                       'borderColor': '#115692',
                       'shadowColor': '#115692',
                       'shadowBlur': 20,
                       'opacity': 0.6
                   }
})

mp.set_global_opts(
    title_opts=opts.TitleOpts(title="Cumulative confirmed global", 
                              subtitle='Update date：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global_confirm.columns[-1], '%m/%d/%y'))),
                              pos_top='5%', pos_left='5%',
                              subtitle_textstyle_opts=opts.TextStyleOpts(font_size=12),
                              title_textstyle_opts=opts.TextStyleOpts(font_size=25,font_weight='bold')),
    legend_opts=opts.LegendOpts(is_show=False),
)

"""
geo：
show some country data like USA and something else
"""
geo = Geo()
geo.add_schema(
    maptype="world",
    is_roam=False,
    zoom=1.2,
    itemstyle_opts={
        'normal': {
            'opacity': 0}})

geo.add("", data_pair_high, type_='effectScatter', symbol_size=JsCode("""function (val) {return val[2] / 2e5;}"""))
geo.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=True,
        position='right',
        color='white',
        font_size=12,
        font_weight='bold',
        formatter=JsCode(fmt_js))
        )

# combine three maps as one
grid = (
    Grid(init_opts=opts.InitOpts(theme='chalk', width='1000px', height='600px'))
    .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_right='70%', pos_left='5%'))
    .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
    .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
)

grid.render_notebook()

### Cumulative death global

In [6]:
df_global_death = pd.read_csv('C:/Users/MinkeyJiang/Desktop/722/old_database/time_series_covid19_deaths_global.csv')

df_global_death.iloc[:,1] = df_global_death.iloc[:,1].apply(replace_country_name)

df_global_country_death = df_global_death.groupby(['Country/Region']).sum().reset_index()

In [7]:
geo = Geo()
data_pair, data_pair_high = [], []
for idx, row in df_global_death.iloc[:,[0,1,2,3,-1]].iterrows():
    if int(row[-1]) > 3e4:
        label = row['Country/Region'] if row['Province/State'] is np.nan  else row['Province/State']
        geo.add_coordinate(label, row['Long'], row['Lat'])
        data_pair_high.append([label, int(row[-1])])
        
for idx, row in df_global_country_death.iloc[:,[0,-1]].iterrows():
    data_pair.append([row['Country/Region'], int(row[-1])])

In [8]:
"""
Bar:
show the daliy top 10 country
leftbottom
"""
t = sorted(data_pair, key=lambda x: x[1], reverse=True)[:10][::-1]

bar = Bar()
bar.add_xaxis([x for x, y in t])
bar.add_yaxis("", [y for x, y in t],
              itemstyle_opts={'normal': {'borderColor': '#115692',
                                         'shadowColor': '#115692',
                                         'shadowBlur': 20,
                                         'opacity': 0.8
                                         }
                              })
# Label format setting
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=10,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
# Global format setting
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, max_=3e5),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="TOP 10", pos_top='55%', pos_left='5%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=5e4,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['rgba(219,112,147,0.4)', 'rgba(238,25,27,0.9)', 'rgba(238,25,27,1)']))
bar.reversal_axis()

"""
map
"""
mp = Map()
mp.add(
    "Cumulative death global",
    data_pair,
    "world",
    zoom=1.2,
    is_map_symbol_show=False,
    is_roam=False)
mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                   itemstyle_opts={'normal': {
                       'areaColor': '#071126',
                       'borderColor': '#115692',
                       'shadowColor': '#115692',
                       'shadowBlur': 20,
                       'opacity': 0.6
                   }
})

mp.set_global_opts(
    title_opts=opts.TitleOpts(title="Cumulative death global", 
                              subtitle='Update date：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global_death.columns[-1], '%m/%d/%y'))),
                              pos_top='5%', pos_left='5%',
                              subtitle_textstyle_opts=opts.TextStyleOpts(font_size=10),
                              title_textstyle_opts=opts.TextStyleOpts(font_size=25,font_weight='bold')),
    legend_opts=opts.LegendOpts(is_show=False),
)

"""
geo：
show some country data like USA and something else
"""
geo = Geo()
geo.add_schema(
    maptype="world",
    is_roam=False,
    zoom=1.2,
    itemstyle_opts={
        'normal': {
            'opacity': 0}})

geo.add("", data_pair_high, type_='effectScatter', symbol_size=JsCode("""function (val) {return val[2] / 1e4;}"""))

geo.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=True,
        position='right',
        color='white',
        font_size=12,
        font_weight='bold',
        formatter=JsCode(fmt_js))
        )
# combine three maps as one
grid = (
    Grid(init_opts=opts.InitOpts(theme='chalk', width='1000px', height='600px'))
    .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_right='70%', pos_left='5%'))
    .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
    .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
)

grid.render_notebook()

## Cumulative recovered global

In [9]:
df_global_recover = pd.read_csv('C:/Users/MinkeyJiang/Desktop/722/old_database/time_series_covid19_recovered_global.csv')

df_global_recover.iloc[:,1] = df_global_recover.iloc[:,1].apply(replace_country_name)

df_global_country_recover = df_global_recover.groupby(['Country/Region']).sum().reset_index()

In [10]:
geo = Geo()
data_pair, data_pair_high = [], []
for idx, row in df_global_recover.iloc[:,[0,1,2,3,-1]].iterrows():
    if int(row[-1]) > 5e5:
        label = row['Country/Region'] if row['Province/State'] is np.nan  else row['Province/State']
        geo.add_coordinate(label, row['Long'], row['Lat'])
        data_pair_high.append([label, int(row[-1])])
        
for idx, row in df_global_country_recover.iloc[:,[0,-1]].iterrows():
    data_pair.append([row['Country/Region'], int(row[-1])])

In [11]:
"""
Bar:
show the daliy top 10 country
leftbottom
"""
t = sorted(data_pair, key=lambda x: x[1], reverse=True)[:10][::-1]

bar = Bar()
bar.add_xaxis([x for x, y in t])
bar.add_yaxis("", [y for x, y in t],
              itemstyle_opts={'normal': {'borderColor': '#115692',
                                         'shadowColor': '#115692',
                                         'shadowBlur': 20,
                                         'opacity': 0.8
                                         }
                              })
bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=10,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))
bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False, max_=5e6),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="TOP 10", 
                                              pos_top='55%', 
                                              pos_left='5%',
                                              title_textstyle_opts=opts.TextStyleOpts(font_size=12)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=5e5,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['#ffffff', 'rgba(0,128,0,1)']))

bar.reversal_axis()

"""
map
"""
mp = Map()
mp.add(
    "Cumulative recovered global",
    data_pair,
    "world",
    zoom=1.2,
    is_map_symbol_show=False,
    is_roam=False)

mp.set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                   itemstyle_opts={'normal': {
                       'areaColor': '#ffffff',
                       'borderColor': '#115692',
                       'shadowColor': '#115692',
                       'shadowOffsetY': 1,
                       'shadowOffsetX': 1,
                       'shadowBlur': 0,
                       'opacity': 1
                   }
})

mp.set_global_opts(
    title_opts=opts.TitleOpts(title="Cumulative recovered global", 
                              subtitle='Update date：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global_recover.columns[-1], '%m/%d/%y'))),
                              pos_top='5%', pos_left='5%',
                              subtitle_textstyle_opts=opts.TextStyleOpts(font_size=10),
                              title_textstyle_opts=opts.TextStyleOpts(font_size=25, font_weight='bold')), legend_opts=opts.LegendOpts(is_show=False),
)

"""
geo：
show some country data like USA and something else
"""
geo = Geo()

geo.add_schema(
    maptype="world",
    is_roam=False,
    zoom=1.2,
    itemstyle_opts={
        'normal': {
            'opacity': 0}})

geo.add("", 
        data_pair_high, 
        type_='scatter', 
        symbol_size=1)

geo.set_series_opts(
    label_opts=opts.LabelOpts(
        is_show=True,
        position='right',
        color='#F5F5F5',
        font_size=13,
        font_weight='bold',
        formatter=JsCode(fmt_js))
        )

# combine three maps as one
grid = (
    Grid(init_opts=opts.InitOpts(theme='dark', width='1000px', height='600px'))
    .add(bar, grid_opts=opts.GridOpts(pos_top="60%", pos_right='70%', pos_left='5%'))
    .add(mp, grid_opts=opts.GridOpts(pos_top="12%"))
    .add(geo, grid_opts=opts.GridOpts(pos_bottom="12%"))
)

grid.render_notebook()

## Chronological order

In [12]:
df_global = pd.merge(df_global_confirm.iloc[:,4:].sum().reset_index(), 
                     df_global_death.iloc[:,4:].sum().reset_index(), 
                     on='index')
df_global = pd.merge(df_global, 
                     df_global_recover.iloc[:,4:].sum().reset_index(), 
                     on='index')
df_global.columns = ['date', 'confirm', 'death', 'recover']

# change data type to list and format date as yyyy-MM-dd
data_x, data_y_confirm, data_y_death, data_y_recover = [],[],[],[]
for idx, row in df_global.iterrows():
    label = time.strftime("%Y-%m-%d",time.strptime(row['date'], '%m/%d/%y'))
    data_x.append(label)
    data_y_confirm.append(int(row['confirm']))
    data_y_death.append(int(row['death']))
    data_y_recover.append(int(row['recover']))


In [13]:
# format output
msg = "['{}:', 'Confirmed：{:,}', 'Death：{:,}','Recovered：{:,}'].join('\\n\\n')"

msg = msg.format(time.strftime("%Y-%m-%d",time.strptime(df_global.iloc[-1,0], '%m/%d/%y')),
                 df_global.iloc[-1, 1],
                 df_global.iloc[-1, 2],
                 df_global.iloc[-1, 3])


In [14]:
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: 'rgba(25,25,25, 0.8)'}, {offset: 1, color: 'rgba(200,139,0,0.5)'}], false)"
)

line_style = {
    'normal': {
        'width': 4,
        'shadowColor': 'rgba(155, 18, 184, .5)',
        'shadowBlur': 10,
        'shadowOffsetY': 20,
        'shadowOffsetX': 20,
        'curve':0.5
    }
}


line = Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js), width='980px', height='600px'))
line.add_xaxis(data_x)
line.add_yaxis("Cumulative confirmed", 
        data_y_confirm, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("Cumulative death", 
        data_y_death, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("Cumulative recovered", 
        data_y_recover, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.set_series_opts(linestyle_opts=line_style,
                     label_opts=opts.LabelOpts(is_show=False, position="top", color="white"),
                     tooltip_opts=opts.TooltipOpts(is_show=True))
line.set_global_opts(title_opts=opts.TitleOpts(title="Global COVID-19",
                                               subtitle='Update date：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global.iloc[-1,0], '%m/%d/%y'))),
                                               pos_left="center",
                                               title_textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                       font_size=20)),
                     legend_opts=opts.LegendOpts(is_show=True, pos_top='2%', 
                                                 pos_left="70%", 
                                                 selected_mode='multiple',
                                                 legend_icon='circle',
                                                 textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                   font_size=10)),
                     xaxis_opts=opts.AxisOpts(boundary_gap=False,
                                                axislabel_opts=opts.LabelOpts(margin=30, 
                                                                              color="#ffffff63",
                                                                              ),
                                                axisline_opts=opts.AxisLineOpts(is_show=False),
                                                axistick_opts=opts.AxisTickOpts(
                                                    is_show=True,
                                                    length=25,
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                                ),
                                                splitline_opts=opts.SplitLineOpts(
                                                    is_show=True, 
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                                ),
                                            ),
                    yaxis_opts=opts.AxisOpts(
                                            type_="value",
                                            position="right",
                                            axislabel_opts=opts.LabelOpts(margin=20, 
                                                                          color="#ffffff63",
                                                                          formatter=JsCode("""function (value) 
                                                                               {return value;}""")),
                                            axisline_opts=opts.AxisLineOpts(
                                                linestyle_opts=opts.LineStyleOpts(width=2, 
                                                                                  color="#fff")
                                            ),
                                            axistick_opts=opts.AxisTickOpts(
                                                is_show=True,
                                                length=15,
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                            ),
                                            splitline_opts=opts.SplitLineOpts(
                                                is_show=True, 
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                            )
                                        ),
                     graphic_opts=[opts.GraphicGroup(
                                        graphic_item=opts.GraphicItem(
                                            bounding="raw",
                                            left=250,
                                            top=150,
                                            z=100,
                                        ),
                                        children=[
                                            opts.GraphicRect(
                                                graphic_item=opts.GraphicItem(
                                                    left="center", top="center", z=100
                                                ),
                                                graphic_shape_opts=opts.GraphicShapeOpts(
                                                    width=200, height=100
                                                ),
                                                graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                                    fill="rgba(0,0,0,0.3)"
                                                ),
                                            ),
                                            opts.GraphicText(
                                                graphic_item=opts.GraphicItem(
                                                    left="center", top="center", z=100
                                                ),
                                                graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                                                    text=JsCode(msg),
                                                    font="bold 12px Microsoft YaHei",
                                                    graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                                        fill="#fff"
                                                    ),
                                                ),
                                            ),
                                        ],
                                    )
                                ],
                    )      
line.render_notebook()

## daily trends

In [15]:
data_x, data_y_confirm_add, data_y_death_add, data_y_recover_add = [],[],[],[]
for idx in range(1, df_global.shape[0]):
    label = time.strftime("%Y-%m-%d",time.strptime(df_global.iloc[idx, 0], '%m/%d/%y'))
    data_x.append(label)
    data_y_confirm_add.append(int(df_global.iloc[idx, 1] - df_global.iloc[idx-1, 1]))
    data_y_death_add.append(int(df_global.iloc[idx, 2] - df_global.iloc[idx-1, 2]))
    data_y_recover_add.append(int(df_global.iloc[idx, 3] - df_global.iloc[idx-1, 3]))

In [16]:
msg = "['{}:', 'New confirmed：{:,}', 'New death：{:,}','New recovered：{:,}'].join('\\n\\n')"

msg = msg.format(time.strftime("%Y-%m-%d",time.strptime(df_global.iloc[-1,0], '%m/%d/%y')),
                 data_y_confirm_add[-1],
                 data_y_death_add[-1],
                 data_y_recover_add[-1])



In [17]:
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: 'rgba(25,25,25, 0.8)'}, {offset: 1, color: 'rgba(200,139,0,0.5)'}], false)"
)

line_style = {
    'normal': {
        'width': 4,
        'shadowColor': 'rgba(155, 18, 184, .5)',
        'shadowBlur': 10,
        'shadowOffsetY': 20,
        'shadowOffsetX': 20,
        'curve':0.5
    }
}


line = Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js), width='980px', height='600px'))
line.add_xaxis(data_x)
line.add_yaxis("New confirmed", 
        data_y_confirm_add, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("New death", 
        data_y_death_add, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("New recovered", 
        data_y_recover_add, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.set_series_opts(linestyle_opts=line_style,
                     label_opts=opts.LabelOpts(is_show=False, position="top", color="white"),
                     tooltip_opts=opts.TooltipOpts(is_show=True))
line.set_global_opts(title_opts=opts.TitleOpts(title="New trend of global epidemic situation",
                                               subtitle='Update date：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global.iloc[-1,0], '%m/%d/%y'))),
                                               pos_left="center",
                                               title_textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                       font_size=20)),
                     legend_opts=opts.LegendOpts(is_show=True, pos_top='2%', 
                                                 pos_left="70%", 
                                                 selected_mode='multiple',
                                                 legend_icon='circle',
                                                 textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                   font_size=10)),
                     xaxis_opts=opts.AxisOpts(boundary_gap=False,
                                                axislabel_opts=opts.LabelOpts(margin=30, 
                                                                              color="#ffffff63",
                                                                              ),
                                                axisline_opts=opts.AxisLineOpts(is_show=False),
                                                axistick_opts=opts.AxisTickOpts(
                                                    is_show=True,
                                                    length=25,
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                                ),
                                                splitline_opts=opts.SplitLineOpts(
                                                    is_show=True, 
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                                ),
                                            ),
                    yaxis_opts=opts.AxisOpts(
                                            type_="value",
                                            position="right",
                                            axislabel_opts=opts.LabelOpts(margin=20, 
                                                                          color="#ffffff63",
                                                                          formatter=JsCode("""function (value) 
                                                                               {return value;}""")),
                                            axisline_opts=opts.AxisLineOpts(
                                                linestyle_opts=opts.LineStyleOpts(width=2, 
                                                                                  color="#fff")
                                            ),
                                            axistick_opts=opts.AxisTickOpts(
                                                is_show=True,
                                                length=15,
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                            ),
                                            splitline_opts=opts.SplitLineOpts(
                                                is_show=True, 
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                            )
                                        ),
                     graphic_opts=[opts.GraphicGroup(
                                        graphic_item=opts.GraphicItem(
                                            bounding="raw",
                                            left=250,
                                            top=150,
                                            z=100,
                                        ),
                                        children=[
                                            opts.GraphicRect(
                                                graphic_item=opts.GraphicItem(
                                                    left="center", top="center", z=100
                                                ),
                                                graphic_shape_opts=opts.GraphicShapeOpts(
                                                    width=200, height=100
                                                ),
                                                graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                                    fill="rgba(0,0,0,0.3)"
                                                ),
                                            ),
                                            opts.GraphicText(
                                                graphic_item=opts.GraphicItem(
                                                    left="center", top="center", z=100
                                                ),
                                                graphic_textstyle_opts=opts.GraphicTextStyleOpts(
                                                    text=JsCode(msg),
                                                    font="bold 12px Microsoft YaHei",
                                                    graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                                        fill="#fff"
                                                    ),
                                                ),
                                            ),
                                        ],
                                    )
                                ],
                    )      
line.render_notebook()

## Increase top 20

In [18]:
df_global_country_confirm['last_day_add'] = df_global_country_confirm.iloc[:, -1] - df_global_country_confirm.iloc[:, -2]

data_new = df_global_country_confirm.sort_values(by='last_day_add', ascending=False)

bar = Bar(init_opts=opts.InitOpts(theme='chalk' ,width='1000px', height='800px'))
bar.add_xaxis(data_new['Country/Region'][:20].tolist()[::-1])
bar.add_yaxis("", data_new['last_day_add'][:20].tolist()[::-1],
              itemstyle_opts={'normal': {'borderColor': '#fff',
                                         'borderWidth': 2,
                                         'shadowBlur': 1,
                                         'opacity': 0.8
                                         }
                              })

bar.set_series_opts(label_opts=opts.LabelOpts(position="insideLeft",
                                              font_size=12,
                                              font_weight='bold',
                                              formatter='{b}:{c}'))

bar.set_global_opts(xaxis_opts=opts.AxisOpts(is_show=False),
                    yaxis_opts=opts.AxisOpts(is_show=False),
                    title_opts=opts.TitleOpts(title="Increase TOP 20", 
                                  subtitle='Update date：{}'.format(time.strftime("%Y-%m-%d",time.strptime(df_global.iloc[-1,0], '%m/%d/%y'))),
                                  pos_top='80%', pos_left='70%',
                                  title_textstyle_opts=opts.TextStyleOpts(font_size=28)),
                    visualmap_opts=opts.VisualMapOpts(is_show=False,
                                                      max_=2e5,
                                                      min_=200,
                                                      is_piecewise=False,
                                                      dimension=0,
                                                      range_color=['rgba(238,25,27,1)', 'rgba(219,112,147,0.4)']),
                    graphic_opts=[
                            opts.GraphicGroup(
                                graphic_item=opts.GraphicItem(left="5%", top="5%"),
                                children=[
                                    opts.GraphicRect(
                                        graphic_item=opts.GraphicItem(
                                            z=0, left="center", top="middle"
                                        ),
                                        graphic_shape_opts=opts.GraphicShapeOpts(width=900, height=720),
                                        graphic_basicstyle_opts=opts.GraphicBasicStyleOpts(
                                            fill="rgba(0,0,0,0)",
                                            stroke="#fff",
                                            line_width=2,
                                            shadow_blur=20,
                                            shadow_offset_x=5,
                                            shadow_offset_y=5,
                                            shadow_color="#4169E1",
                                        ),
                                    ),
                                ],
                            )
                        ]
                    )
bar.reversal_axis()
bar.render_notebook()

## US's COVID-19 data

In [19]:
# us data
df_total = pd.read_csv('C:/Users/MinkeyJiang/Desktop/722/old_database/us-data.csv')

## US's COVID-19

In [20]:
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: 'rgba(25,25,25, 0.8)'}, {offset: 1, color: 'rgba(200,139,0,0.5)'}], false)"
)

line_style = {
    'normal': {
        'width': 4,
        'shadowColor': 'rgba(155, 18, 184, .5)',
        'shadowBlur': 10,
        'shadowOffsetY': 20,
        'shadowOffsetX': 20,
        'curve':0.5
    }
}

line = Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js), width='960px', height='600px'))

line.add_xaxis(df_total.date.tolist())

line.add_yaxis("Cumulative confirmed", 
        df_total.cases.tolist(), 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )
line.add_yaxis("Cumulative death", 
        df_total.deaths.tolist(), 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )
line.add_yaxis("Cumulative recovered", 
        df_total.recovered.tolist(), 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.set_series_opts(linestyle_opts=line_style,
                     label_opts=opts.LabelOpts(is_show=False, position="top", color="white"),
                     tooltip_opts=opts.TooltipOpts(is_show=True))
line.set_global_opts(title_opts=opts.TitleOpts(title="US's COVID-19",
                                               subtitle='Update date：{}'.format(df_total.date.max()),
                                               pos_left="center",
                                               title_textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                       font_size=20)),
                     legend_opts=opts.LegendOpts(is_show=True, pos_top='2%', 
                                                 pos_left="70%", 
                                                 selected_mode='multiple',
                                                 legend_icon='circle',
                                                 textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                   font_size=10)),
                     xaxis_opts=opts.AxisOpts(boundary_gap=False,
                                                axislabel_opts=opts.LabelOpts(margin=30, 
                                                                              color="#ffffff63",
                                                                              ),
                                                axisline_opts=opts.AxisLineOpts(is_show=False),
                                                axistick_opts=opts.AxisTickOpts(
                                                    is_show=True,
                                                    length=25,
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                                ),
                                                splitline_opts=opts.SplitLineOpts(
                                                    is_show=True, 
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                                ),
                                            ),
                    yaxis_opts=opts.AxisOpts(
                                            type_="value",
                                            position="right",
                                            axislabel_opts=opts.LabelOpts(margin=20, 
                                                                          color="#ffffff63",
                                                                          formatter=JsCode("""function (value) 
                                                                               {return value;}""")),
                                            axisline_opts=opts.AxisLineOpts(
                                                linestyle_opts=opts.LineStyleOpts(width=2, 
                                                                                  color="#fff")
                                            ),
                                            axistick_opts=opts.AxisTickOpts(
                                                is_show=True,
                                                length=15,
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                            ),
                                            splitline_opts=opts.SplitLineOpts(
                                                is_show=True, 
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                            )
                                        ),
                    )        
line.render_notebook()

## US daily trends

In [21]:
data_x, data_y_1, data_y_2, data_y_3 = [], [], [], []
for i in range(1, df_total.shape[0]):
    data_x.append(df_total.iloc[i, 0])
    data_y_1.append(int(df_total.iloc[i, 1] - df_total.iloc[i-1, 1]))
    data_y_2.append(int(df_total.iloc[i, 2] - df_total.iloc[i-1, 2]))
    data_y_3.append(int(df_total.iloc[i, 3] - df_total.iloc[i-1, 3]))
    
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: 'rgba(25,25,25, 0.8)'}, {offset: 1, color: 'rgba(200,139,0,0.5)'}], false)"
)

line_style = {
    'normal': {
        'width': 4,
        'shadowColor': 'rgba(155, 18, 184, .5)',
        'shadowBlur': 10,
        'shadowOffsetY': 20,
        'shadowOffsetX': 20,
        'curve':0.5
    }
}


line = Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js), width='980px', height='600px'))
line.add_xaxis(data_x)
line.add_yaxis("New confirmed", 
        data_y_1, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("New death", 
        data_y_2, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("New recovered", 
        data_y_3, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.set_series_opts(linestyle_opts=line_style,
                     label_opts=opts.LabelOpts(is_show=False, position="top", color="white"),
                     tooltip_opts=opts.TooltipOpts(is_show=True))
line.set_global_opts(title_opts=opts.TitleOpts(title="US daily trends",
                                               subtitle='Update date：{}'.format(df_total.date.max()),
                                               pos_left="center",
                                               title_textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                       font_size=20)),
                     legend_opts=opts.LegendOpts(is_show=True, pos_top='2%', 
                                                 pos_left="70%", 
                                                 selected_mode='multiple',
                                                 legend_icon='circle',
                                                 textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                   font_size=10)),
                     xaxis_opts=opts.AxisOpts(boundary_gap=False,
                                                axislabel_opts=opts.LabelOpts(margin=30, 
                                                                              color="#ffffff63",
                                                                              ),
                                                axisline_opts=opts.AxisLineOpts(is_show=False),
                                                axistick_opts=opts.AxisTickOpts(
                                                    is_show=True,
                                                    length=25,
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                                ),
                                                splitline_opts=opts.SplitLineOpts(
                                                    is_show=True, 
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                                ),
                                            ),
                    yaxis_opts=opts.AxisOpts(
                                            type_="value",
                                            position="right",
                                            axislabel_opts=opts.LabelOpts(margin=20, 
                                                                          color="#ffffff63",
                                                                          formatter=JsCode("""function (value) 
                                                                               {return value;}""")),
                                            axisline_opts=opts.AxisLineOpts(
                                                linestyle_opts=opts.LineStyleOpts(width=2, 
                                                                                  color="#fff")
                                            ),
                                            axistick_opts=opts.AxisTickOpts(
                                                is_show=True,
                                                length=15,
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                            ),
                                            splitline_opts=opts.SplitLineOpts(
                                                is_show=True, 
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                            )
                                        ),
                    )      
line.render_notebook()

## China’s COVID-19 data

In [22]:
# China data
df_total = pd.read_csv('C:/Users/MinkeyJiang/Desktop/722/old_database/China-data.csv')

## China's COVID-19

In [23]:
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: 'rgba(25,25,25, 0.8)'}, {offset: 1, color: 'rgba(200,139,0,0.5)'}], false)"
)

line_style = {
    'normal': {
        'width': 4,
        'shadowColor': 'rgba(155, 18, 184, .5)',
        'shadowBlur': 10,
        'shadowOffsetY': 20,
        'shadowOffsetX': 20,
        'curve':0.5
    }
}


line = Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js), width='960px', height='600px'))

line.add_xaxis(df_total.date.tolist())

line.add_yaxis("Cumulative confirmed", 
        df_total.cases.tolist(), 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )
line.add_yaxis("Cumulative death", 
        df_total.deaths.tolist(), 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )
line.add_yaxis("Cumulative recovered", 
        df_total.recovered.tolist(), 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.set_series_opts(linestyle_opts=line_style,
                     label_opts=opts.LabelOpts(is_show=False, position="top", color="white"),
                     tooltip_opts=opts.TooltipOpts(is_show=True))
line.set_global_opts(title_opts=opts.TitleOpts(title="China's COVID-19",
                                               subtitle='Update date：{}'.format(df_total.date.max()),
                                               pos_left="center",
                                               title_textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                       font_size=20)),
                     legend_opts=opts.LegendOpts(is_show=True, pos_top='2%', 
                                                 pos_left="70%", 
                                                 selected_mode='multiple',
                                                 legend_icon='circle',
                                                 textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                   font_size=10)),
                     xaxis_opts=opts.AxisOpts(boundary_gap=False,
                                                axislabel_opts=opts.LabelOpts(margin=30, 
                                                                              color="#ffffff63",
                                                                              ),
                                                axisline_opts=opts.AxisLineOpts(is_show=False),
                                                axistick_opts=opts.AxisTickOpts(
                                                    is_show=True,
                                                    length=25,
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                                ),
                                                splitline_opts=opts.SplitLineOpts(
                                                    is_show=True, 
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                                ),
                                            ),
                    yaxis_opts=opts.AxisOpts(
                                            type_="value",
                                            position="right",
                                            axislabel_opts=opts.LabelOpts(margin=20, 
                                                                          color="#ffffff63",
                                                                          formatter=JsCode("""function (value) 
                                                                               {return value;}""")),
                                            axisline_opts=opts.AxisLineOpts(
                                                linestyle_opts=opts.LineStyleOpts(width=2, 
                                                                                  color="#fff")
                                            ),
                                            axistick_opts=opts.AxisTickOpts(
                                                is_show=True,
                                                length=15,
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                            ),
                                            splitline_opts=opts.SplitLineOpts(
                                                is_show=True, 
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                            )
                                        ),
                    )      
line.render_notebook()

## China daily trends

In [24]:
data_x, data_y_1, data_y_2, data_y_3 = [], [], [], []
for i in range(1, df_total.shape[0]):
    data_x.append(df_total.iloc[i, 0])
    data_y_1.append(int(df_total.iloc[i, 1] - df_total.iloc[i-1, 1]))
    data_y_2.append(int(df_total.iloc[i, 2] - df_total.iloc[i-1, 2]))
    data_y_3.append(int(df_total.iloc[i, 3] - df_total.iloc[i-1, 3]))
background_color_js = (
    "new echarts.graphic.LinearGradient(0, 0, 0, 1, "
    "[{offset: 0, color: 'rgba(25,25,25, 0.8)'}, {offset: 1, color: 'rgba(200,139,0,0.5)'}], false)"
)

line_style = {
    'normal': {
        'width': 4,
        'shadowColor': 'rgba(155, 18, 184, .5)',
        'shadowBlur': 10,
        'shadowOffsetY': 20,
        'shadowOffsetX': 20,
        'curve':0.5
    }
}


line = Line(init_opts=opts.InitOpts(bg_color=JsCode(background_color_js), width='980px', height='600px'))
line.add_xaxis(data_x)
line.add_yaxis("New confirmed", 
        data_y_1, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("New death", 
        data_y_2, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.add_yaxis("New recovered", 
        data_y_3, 
        is_smooth=True, 
        is_connect_nones=True,
        symbol="circle",
        symbol_size=6,
        itemstyle_opts=opts.ItemStyleOpts(
                border_color="#fff", border_width=3),
        )

line.set_series_opts(linestyle_opts=line_style,
                     label_opts=opts.LabelOpts(is_show=False, position="top", color="white"),
                     tooltip_opts=opts.TooltipOpts(is_show=True))
line.set_global_opts(title_opts=opts.TitleOpts(title="China daily trends",
                                               subtitle='Update date：{}'.format(df_total.date.max()),
                                               pos_left="center",
                                               title_textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                       font_size=20)),
                     legend_opts=opts.LegendOpts(is_show=True, pos_top='2%', 
                                                 pos_left="70%", 
                                                 selected_mode='multiple',
                                                 legend_icon='circle',
                                                 textstyle_opts=opts.TextStyleOpts(color="white", 
                                                                                   font_size=10)),
                     xaxis_opts=opts.AxisOpts(boundary_gap=False,
                                                axislabel_opts=opts.LabelOpts(margin=30, 
                                                                              color="#ffffff63",
                                                                              ),
                                                axisline_opts=opts.AxisLineOpts(is_show=False),
                                                axistick_opts=opts.AxisTickOpts(
                                                    is_show=True,
                                                    length=25,
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                                ),
                                                splitline_opts=opts.SplitLineOpts(
                                                    is_show=True, 
                                                    linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                                ),
                                            ),
                    yaxis_opts=opts.AxisOpts(
                                            type_="value",
                                            position="right",
                                            axislabel_opts=opts.LabelOpts(margin=20, 
                                                                          color="#ffffff63",
                                                                          formatter=JsCode("""function (value) 
                                                                               {return value;}""")),
                                            axisline_opts=opts.AxisLineOpts(
                                                linestyle_opts=opts.LineStyleOpts(width=2, 
                                                                                  color="#fff")
                                            ),
                                            axistick_opts=opts.AxisTickOpts(
                                                is_show=True,
                                                length=15,
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f"),
                                            ),
                                            splitline_opts=opts.SplitLineOpts(
                                                is_show=True, 
                                                linestyle_opts=opts.LineStyleOpts(color="#ffffff1f")
                                            )
                                        ),
                    )      
line.render_notebook()